# Getting started processing Kafka with Spark

The first thing we'll need to do is tell Spark where to find the Kafka driver before we set Spark up.  Currently, our notebook images are built against Spark 2.2.  If you're using this with a different version of Spark, be sure to change `SPARK_VERSION` in the cell below before executing it.

In [1]:
import os
SPARK_VERSION="2.2.0"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages org.apache.spark:spark-sql-kafka-0-10_2.11:%s pyspark-shell" % SPARK_VERSION

Next up, we'll connect to Spark by establishing a `SparkSession`.

In [2]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Social Firehose") \
    .getOrCreate()

We're going to begin by loading the contents of a Kafka topic into a data frame.  Because Spark data frames are _lazy_, or recomputed when accessed, this data frame will always have the most recent collection of messages in it.

In [3]:
df = spark \
  .read \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc:9092") \
  .option("subscribe", "social-firehose") \
  .load()

We can see that this data frame always has the most recent collection of messages by running the `count()` action on it twice with a short delay in the middle.  Note how many messages are generated in ten seconds:

In [4]:
import time
a = df.count()
time.sleep(10)
b = df.count()
(a, b)

(1026698, 1026716)

We can inspect the first few messages, but they'll be in a pretty raw format.

In [5]:
df.take(3)

[Row(key=None, value=bytearray(b'{"text": "The furniture was in all probability have gained some news of them; and till we know that she ever should receive another so perfectly gratifying in the occasion and the style. #retweet #yolo #ff", "user_id": "9086078734", "update_id": "00000000000000000000"}'), topic='social-firehose', partition=0, offset=0, timestamp=datetime.datetime(2018, 5, 1, 13, 55, 35, 78000), timestampType=0),
 Row(key=None, value=bytearray(b'{"text": "After this period every appearance of equal permanency. #health", "user_id": "3082369400", "update_id": "00000000000000000001"}'), topic='social-firehose', partition=0, offset=1, timestamp=datetime.datetime(2018, 5, 1, 13, 55, 35, 273000), timestampType=0),
 Row(key=None, value=bytearray(b'{"text": "Worse than all! #health #news", "user_id": "7761320665", "update_id": "00000000000000000002"}'), topic='social-firehose', partition=0, offset=2, timestamp=datetime.datetime(2018, 5, 1, 13, 55, 35, 343000), timestampType=0)]

Now we'll import some functions and types from the Spark library so we can do something more useful with our data set.

In [6]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import from_json
from pyspark.sql.functions import column
from pyspark.sql.types import StringType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

The first thing we'll do is extract the JSON payloads of the messages; we'll inspect the first ten as a sanity check.

In [7]:
values = df.select(df["value"].cast(StringType()).alias("value"))
values.show(10)

+--------------------+
|               value|
+--------------------+
|{"text": "The fur...|
|{"text": "After t...|
|{"text": "Worse t...|
|{"text": "She is ...|
|{"text": "Miss #C...|
|{"text": "Where p...|
|{"text": "Miss #H...|
|{"text": "Why the...|
|{"text": "The ent...|
|{"text": "Emma co...|
+--------------------+
only showing top 10 rows



The next thing we'll do is impose some structure on the messages by converting the serialized JSON objects into actual records:

1.  First, we'll declare a `StructType` for the structure of our messages (three strings, named `text`, `user_id`, and `update_id`),
2.  Next, we'll convert the JSON strings to structures using the `from_json` dataframe function, and
3.  Finally, we'll `SELECT` the fields of the object so we have something that looks like a flat database tuple.

In [8]:
structure = StructType([StructField(fn, StringType(), True) for fn in "text user_id update_id".split()])
records = values.select(from_json(values["value"], structure).alias("json")) \
                .select(column("json.update_id"), column("json.user_id").alias("user_id"), column("json.text"))
records.show(10)

+--------------------+----------+--------------------+
|           update_id|   user_id|                text|
+--------------------+----------+--------------------+
|00000000000000000000|9086078734|The furniture was...|
|00000000000000000001|3082369400|After this period...|
|00000000000000000002|7761320665|Worse than all! #...|
|00000000000000000003|2529702535|She is netting he...|
|00000000000000000004|5580232535|Miss #Crawford wa...|
|00000000000000000005|2143036217|Where pride and s...|
|00000000000000000006|1605193990|Miss #Hamilton, n...|
|00000000000000000007|1250771648|Why they are your...|
|00000000000000000008|5606455308|The entrance of t...|
|00000000000000000009|1658432974|Emma could not ma...|
+--------------------+----------+--------------------+
only showing top 10 rows



We can perform database-style aggregations on this data frame, like identifying the users responsible for the most status updates:

In [9]:
user_counts = records.groupBy("user_id").count().orderBy("count", ascending=False)
user_counts.show()

+----------+-----+
|   user_id|count|
+----------+-----+
|8804573903| 1088|
|3380222115| 1075|
|0505683347| 1073|
|3000579014| 1069|
|9587912416| 1068|
|4327469857| 1068|
|3108294958| 1066|
|7334906826| 1064|
|0842976912| 1061|
|8396961232| 1060|
|4089612749| 1059|
|9865588092| 1057|
|4612190570| 1055|
|0604401354| 1054|
|3526538619| 1053|
|7029874273| 1053|
|5104535196| 1052|
|8598111212| 1051|
|4165442352| 1051|
|6896171244| 1051|
+----------+-----+
only showing top 20 rows



If you run that query several times with a short delay in between, you may get different results since the data frame will reflect newly-arriving messages.  Try it out!

We can also count the number of users who have issued status updates (because of how we're generating the synthetic stream of updates, there is an upper bound on this number):

In [10]:
records.select("user_id").distinct().count()

19340

We can also identify the most prolix users.  You probably have some social media connections who take advantage of every extra bit of character limit; a query like this will show you who they are!

In [11]:
from pyspark.sql.functions import length
user_loquacity = records.select(column("user_id"), length("text").alias("update_len")) \
  .groupBy("user_id") \
  .avg() \
  .orderBy("avg(update_len)", ascending=False)
user_loquacity.show()

+----------+---------------+
|   user_id|avg(update_len)|
+----------+---------------+
|0630541675|          330.0|
|0391736751|          316.0|
|2506022037|          307.0|
|5821398050|          302.0|
|1410371328|          299.0|
|1412562247|          284.0|
|7790898195|          277.0|
|0223724881|          276.0|
|7578073215|          275.0|
|6407600570|          274.0|
|3076549109|          273.0|
|7999575942|          273.0|
|4408278586|          272.0|
|8725814325|          272.0|
|4764843224|          269.0|
|7652850466|          269.0|
|1036217429|          268.0|
|1432169939|          268.0|
|5449920443|          268.0|
|2160446304|          267.5|
+----------+---------------+
only showing top 20 rows



We can also identify the most popular hashtags in users' updates.  We'll start by turning each update into an array of words.  Then we'll explode each array into multiple rows, so that each row has a separate, single element, i.e.

```
1, 2, "foo bar blah"
```

would become

```
1, 2, [foo, bar, blah]
```

which would become

```
1, 2, foo
1, 2, bar
1, 2, blah
```

We'll then filter for hashtags (keeping only words starting with `#`) so we can find the most popular!


In [12]:
words = records.select(explode(split("text", " ")).alias("word"))
hashtags = words.filter(column("word").startswith("#"))

In [13]:
words.show()

+-----------+
|       word|
+-----------+
|        The|
|  furniture|
|        was|
|         in|
|        all|
|probability|
|       have|
|     gained|
|       some|
|       news|
|         of|
|      them;|
|        and|
|       till|
|         we|
|       know|
|       that|
|        she|
|       ever|
|     should|
+-----------+
only showing top 20 rows



In [14]:
hashtags.show()

+-------------+
|         word|
+-------------+
|     #retweet|
|        #yolo|
|          #ff|
|      #health|
|      #health|
|        #news|
|          #ff|
|    #Crawford|
|        #news|
|        #fail|
|     #retweet|
|#followfriday|
|        #news|
|   #Hamilton,|
|   #MrsSmith,|
|    #MrsSmith|
|    #MrsSmith|
|    #MrsSmith|
|       #Bath,|
|        #Anne|
+-------------+
only showing top 20 rows



In [15]:
hashtag_counts = hashtags.groupBy("word").count().orderBy("count", ascending=False)
hashtag_counts.show()

+----------+-----+
|      word|count|
+----------+-----+
|    #first|15356|
|      #one|13781|
|      #two|11384|
|#Elizabeth|10312|
|    #Fanny| 8513|
|     #Anne| 5927|
|#Catherine| 5719|
| #Marianne| 4842|
|   #Amazon| 4779|
| #Crawford| 4468|
|     #Emma| 4178|
|   #Elinor| 3973|
|     #Jane| 3838|
|   #Weston| 3834|
|    #Darcy| 3793|
|   #Bennet| 3620|
|     #half| 3521|
|    #three| 3477|
|   #second| 3459|
|   #Thomas| 3289|
+----------+-----+
only showing top 20 rows

